In [46]:
import numpy as np
import cv2
import time
import pyautogui
import win32api as wapi
from numpy import ones, vstack
from grabFrame import grab_frame
from numpy.linalg import lstsq
from driveKeys import PressKey, ReleaseKey, W, A, S, D
from statistics import mean

import warnings
# Ignore all warnings and never print.
warnings.filterwarnings('ignore')

In [47]:
def region_of_interest(frame, polygon):
    
    mask = np.zeros_like(frame)   
    cv2.fillPoly(mask, polygon, 255)
    
    # Returning the frame where mask pixels are non-zero
    masked_frame = cv2.bitwise_and(frame, mask)
    return masked_frame

In [48]:
def draw_lanes(frame, lines):

    try:

        ys = []  
        for i in lines:
            for ii in i:
                ys += [ii[1], ii[3]]
        min_y = min(ys)
        max_y = 600
        new_lines = []
        line_dict = {}

        for idx, i in enumerate(lines):
            for xyxy in i:
                x_coords = (xyxy[0], xyxy[2])
                y_coords = (xyxy[1], xyxy[3])
                A = vstack([x_coords, ones(len(x_coords))]).T
                m, b = lstsq(A, y_coords)[0]

                x1 = (min_y - b) / m
                x2 = (max_y - b) / m

                line_dict[idx] = [m,b,[int(x1), min_y, int(x2), max_y]]
                new_lines.append([int(x1), min_y, int(x2), max_y])

        final_lanes = {}

        for idx in line_dict:
            final_lanes_copy = final_lanes.copy()
            m = line_dict[idx][0]
            b = line_dict[idx][1]
            line = line_dict[idx][2]
            
            if len(final_lanes) == 0:
                final_lanes[m] = [ [m, b, line] ]
                
            else:
                found_copy = False

                for other_ms in final_lanes_copy:

                    if not found_copy:
                        if abs(other_ms*1.2) > abs(m) > abs(other_ms*0.8):
                            if abs(final_lanes_copy[other_ms][0][1]*1.2) > abs(b) > abs(final_lanes_copy[other_ms][0][1]*0.8):
                                final_lanes[other_ms].append([m,b,line])
                                found_copy = True
                                break
                        else:
                            final_lanes[m] = [ [m, b, line] ]

        line_counter = {}

        for lanes in final_lanes:
            line_counter[lanes] = len(final_lanes[lanes])

        top_lanes = sorted(line_counter.items(), key = lambda item: item[1])[::-1][:2]

        lane1_id = top_lanes[0][0]
        lane2_id = top_lanes[1][0]

        def average_lane(lane_data):
            x1s = []
            y1s = []
            x2s = []
            y2s = []
            
            for data in lane_data:
                x1s.append(data[2][0])
                y1s.append(data[2][1])
                x2s.append(data[2][2])
                y2s.append(data[2][3])
                
            return int(mean(x1s)), int(mean(y1s)), int(mean(x2s)), int(mean(y2s)) 

        l1_x1, l1_y1, l1_x2, l1_y2 = average_lane(final_lanes[lane1_id])
        l2_x1, l2_y1, l2_x2, l2_y2 = average_lane(final_lanes[lane2_id])

        return [l1_x1, l1_y1, l1_x2, l1_y2], [l2_x1, l2_y1, l2_x2, l2_y2], lane1_id, lane2_id
    
    except Exception as e:
        pass

In [49]:
def bgr_to_hls(frame):
    
    hls_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS)
      
    # defining range for Yellow and White Color
    lower = np.array([0, 153, 153])
    upper = np.array([255, 255, 255])
    
    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(hls_frame, lower, upper)
    
    # Bitwise-AND mask with original frame
    hls_frame_white_yellow = cv2.bitwise_and(frame, frame, mask = mask)
    
    return hls_frame_white_yellow

In [50]:
def contrast_brightness(frame):
    
    contrast_brightness_frame = np.zeros(frame.shape, frame.dtype)
    alpha = 0   # Simple contrast control
    beta = 1    # Simple brightness control
       
    contrast_brightness_frame = cv2.convertScaleAbs(frame, alpha, beta)
    
    return contrast_brightness_frame

In [51]:
def image_processing(frame):
    
    original_frame = frame
    contrast_brightness_frame = contrast_brightness(frame)
    
    hls_frame = bgr_to_hls(contrast_brightness_frame)
    hls_to_bgr_frame = cv2.cvtColor(hls_frame, cv2.COLOR_HLS2BGR)
    
    gray_frame = cv2.cvtColor(hls_to_bgr_frame, cv2.COLOR_BGR2GRAY)
    canny_frame = cv2.Canny(gray_frame, 200, 300)    
    processed_frame = cv2.GaussianBlur(canny_frame, (5, 5), 0)
    
    polygon = np.array([[100, 450], [300, 200], [500, 200], [700, 450], [500, 450], [400, 300], [300, 450]], np.int32)
    processed_frame = region_of_interest(processed_frame, [polygon])
          
    lines = cv2.HoughLinesP(processed_frame, 1, np.pi/180, 180, np.array([]), 50, 10)
    
    m1, m2 = 0, 0
    try:        
        l1, l2, m1, m2 = draw_lanes(original_frame, lines)
        cv2.line(original_frame, (l1[0], l1[1]), (l1[2], l1[3]), [0, 255, 0], 20)
        cv2.line(original_frame, (l2[0], l2[1]), (l2[2], l2[3]), [0, 255, 0], 20)
    
    except Exception as e:
        pass
    
    try:        
        for coords in lines:
            coords = coords[0]
            try:
                cv2.line(processed_frame, (coords[0], coords[1]), (coords[2], coords[3]), [255, 0, 0], 3)
              
            except Exception as e:
                pass
                
    except Exception as e:
        pass

    return hls_to_bgr_frame, processed_frame, original_frame, m1, m2

In [52]:
# Movement
def straight():
    PressKey(W)
    ReleaseKey(A)
    ReleaseKey(S)
    ReleaseKey(D)
    time.sleep(0.06)
    ReleaseKey(W)

def left():
    PressKey(W)
    PressKey(A)
    time.sleep(0.01)
    ReleaseKey(W)
    #ReleaseKey(A)
    PressKey(S)
    ReleaseKey(S) 
    ReleaseKey(D)
    

def right():
    PressKey(W)
    PressKey(D)
    time.sleep(0.01)
    ReleaseKey(W)
    #ReleaseKey(D)
    PressKey(S)
    ReleaseKey(S) 
    ReleaseKey(A)    
    
def slow():
    PressKey(W)
    ReleaseKey(A)
    ReleaseKey(D)
    ReleaseKey(S)
    time.sleep(0.03)
    ReleaseKey(W)

In [53]:
def main():
    
    for i in list(range(5))[::-1]:
        print(i+1)
        time.sleep(1)
    
    last_time = time.time()
    while True:
        frame = grab_frame(region = (0, 40, 800, 640))
        #print('Frame took {} seconds'.format(time.time()-last_time))
        last_time = time.time()
        hls_to_bgr_frame, processed_frame, original_frame, m1, m2 = image_processing(frame)
        
        hls_to_bgr_frame = cv2.resize(hls_to_bgr_frame, (550, 400))
        processed_frame = cv2.resize(processed_frame, (550, 400))
        original_frame = cv2.resize(frame, (550, 400))
                
        #cv2.imshow('HLS Frame', hls_to_bgr_frame)
        cv2.imshow('Lane Lines', processed_frame)
        cv2.imshow('Weighted Frame', cv2.cvtColor(original_frame, cv2.COLOR_BGR2RGB))
              
        if m1 < 0 and m2 < 0:
            right()
            print('Right    : [ {:.4f}, {:.4f} ]'.format(m1, m2))            
        elif m1 > 0 and m2 > 0:
            left()
            print('Left     : [  {:.4f},  {:.4f} ]'.format(m1, m2)) 
        elif m1 < 0 and m2 > 0:
            straight()
            print('Straight : [  {:.4f}, {:.4f} ]'.format(m1, m2))
        else:
            slow()
            print('Slow     : [  {:.4f},  {:.4f} ]'.format(m1, m2)) 
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break

In [55]:
main()

5
4
3
2
1
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  1.0000,  -0.8706 ]
Left     : [  1.0000,  1.1463 ]
Left     : [  1.0000,  0.7500 ]
Left     : [  1.0000,  1.1473 ]
Slow     : [  1.0000,  -0.8706 ]
Straight : [  -0.9048, 1.1485 ]
Slow     : [  1.0000,  -0.8683 ]
Left     : [  1.0000,  0.7522 ]
Slow     : [  1.0000,  -0.8721 ]
Left     : [  1.0000,  1.1511 ]
Slow     : [  1.0000,  -0.8696 ]
Left     : [  1.0000,  1.0000 ]
Left     : [  1.0000,  0.7568 ]
Left     : [  1.0000,  0.4674 ]
Slow     : [  1.0000,  -0.8702 ]
Left     : [  1.0000,  1.1441 ]
Slow     : [  1.0000,  -0.8732 ]
Left     : [  1.0000,  0.7791 ]
Left     : [  1.0000,  0.7759 ]
Straight : [  -0.8689, 0.7527 ]
Left     : [  1.0000,  0.4709 ]
Left     : [  1.0000,  0.7542 ]
Slow     : [  1.0000,  -0.8698 ]
Slow     : [  1.0000,  -0.8982 ]
Slow     : [  1.0000,  -0.8726 ]
Left     : [  1.0000,  0.4464 ]
Slow     : [  1.0000,  -0.9034 ]
Left     : [  1.0000,  0.4675 ]
Slow     : [  1.000

Straight : [  -1.0000, 0.7554 ]
Slow     : [  0.7816,  -1.0000 ]
Straight : [  -1.0000, 0.7581 ]
Straight : [  -1.0000, 0.7813 ]
Straight : [  -1.0000, 0.7083 ]
Slow     : [  0.7820,  -1.0000 ]
Slow     : [  0.7820,  -1.0000 ]
Straight : [  -1.0000, 0.7530 ]
Slow     : [  0.7813,  -1.0000 ]
Straight : [  -1.0000, 0.7842 ]
Straight : [  -1.0000, 0.7824 ]
Straight : [  -1.0000, 0.7831 ]
Straight : [  -1.0000, 0.7826 ]
Slow     : [  0.7791,  -1.0000 ]
Straight : [  -1.0000, 0.7823 ]
Straight : [  -1.0000, 0.7552 ]
Straight : [  -1.0000, 0.7849 ]
Slow     : [  0.7826,  -1.0000 ]
Slow     : [  0.8107,  -1.0000 ]
Straight : [  -1.0000, 0.7824 ]
Slow     : [  0.7811,  -1.0000 ]
Slow     : [  0.7799,  -1.0000 ]
Slow     : [  0.8094,  -1.0000 ]
Straight : [  -1.0000, 0.8090 ]
Straight : [  -1.0000, 0.7538 ]
Straight : [  -1.0000, 0.7826 ]
Straight : [  -1.0000, 0.7812 ]
Straight : [  -1.0000, 0.7270 ]
Slow     : [  0.8099,  -1.0000 ]
Slow     : [  0.7834,  -1.0000 ]
Straight : [  -1.0000, 0.753

Slow     : [  1.0000,  -1.0000 ]
Slow     : [  1.0000,  -1.0000 ]
Slow     : [  1.0000,  -1.0000 ]
Slow     : [  1.0000,  -1.0000 ]
Left     : [  1.0000,  0.4873 ]
Slow     : [  1.0000,  -1.0000 ]
Slow     : [  1.0000,  -1.0000 ]
Slow     : [  1.0000,  -1.0000 ]
Left     : [  1.0000,  0.5079 ]
Left     : [  1.0000,  1.1902 ]
Left     : [  1.0000,  0.5313 ]
Left     : [  1.0000,  1.1449 ]
Left     : [  1.0000,  0.7818 ]
Left     : [  1.0000,  0.9007 ]
Left     : [  1.0000,  1.2203 ]
Left     : [  1.0000,  1.1879 ]
Left     : [  0.9333,  1.2360 ]
Left     : [  1.0000,  0.8358 ]
Left     : [  1.0000,  1.0000 ]
Left     : [  1.0000,  1.0000 ]
Left     : [  1.0000,  1.2330 ]
Left     : [  1.0000,  1.1887 ]
Left     : [  1.2791,  1.0000 ]
Left     : [  1.2791,  1.0000 ]
Left     : [  1.0000,  1.2857 ]
Left     : [  1.0000,  1.2791 ]
Left     : [  1.0000,  1.3253 ]
Left     : [  1.0000,  1.2791 ]
Left     : [  1.2791,  1.0000 ]
Left     : [  1.2346,  1.0000 ]
Left     : [  1.2828,  1.1053 ]
L